# Model Strangelog
Statistical approach to label anomalies in event logs based on percentile analysis of tokenized events.

It returns a labelled dataframe from a test df_trace with new column 'strangeness' for each event:

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

**Requires**
- log dataset from high level task
- LogColor instance pretrained with similar dataset
- A subset column with a partition of similar task executions. For example, TPL_ID.

### Example: traces grouped by TPL_ID

In [1]:
# Init the public dataset
from eliana.datasets import ParlogsObservations
config = {
    'system': 'GRAVITY',
    'period': '1w'
}
logs = ParlogsObservations(**config)

In [2]:
# Partition in success and errors
meta_success = logs.meta[ logs.meta['ERROR'] == False ]
meta_errors  = logs.meta[ logs.meta['ERROR'] == True ]
print(f'In {logs.system}, from {len(logs.meta)} executions, Success: {len(meta_success)}; Error: {len(meta_errors)}')

In GRAVITY, from 152 executions, Success: 128; Error: 24


In [3]:
# Train Log Colorizer
traces = logs.traces()
traces['event'] = traces['logtext']
print(f'Loaded {len(traces)} traces')

Loaded 316207 traces


In [4]:
from eliana.preprocessing import VltLogColorizer
colorizer = VltLogColorizer()
_ = colorizer.fit(traces, warn=True)

## Train Strangelog model

In [5]:
from eliana.models import Strangelog
subset_col='TPL_ID'

model = Strangelog(tokenizer=colorizer, subset_col=subset_col)
model.fit(meta_success, traces, warn=True)

## Predict for SUCCESS executions

SUCCESS should have very low instances of rare (2) and unexpected (3) events

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

In [6]:
for row in meta_success.sample(5).itertuples():
    print(f"Strangeness for {row.TPL_ID} (Error = {row.ERROR})")
    labelled = model.predict(row.TPL_ID, traces[ traces['trace_id'] == row.Index ] )
    display( labelled.groupby('strangeness').size().reset_index(name='count'))


Strangeness for GRAVITY_gen_cal_standby (Error = False)


,strangeness,count
0,0,54
1,1,8


Strangeness for GRAVITY_gen_tec_PTref (Error = False)


,strangeness,count
0,0,268
1,1,1


Strangeness for P2VMWAVESC (Error = False)


,strangeness,count
0,0,3474
1,1,205


Strangeness for P2VMWAVESC (Error = False)


,strangeness,count
0,0,3900
1,1,258
2,2,3


Strangeness for GRAVITY_gen_cal_p2vm (Error = False)


,strangeness,count
0,0,2324
1,1,150


## Predict for ERROR executions

ERRORS should have hign number of rare (2) and unexpected (3) events. And in general, the content of the unexpected should notify about the cause of errors.

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

In [7]:
for row in meta_errors.sample(5).itertuples():
    print(f"-------\nUnexpected events in {row.TPL_ID} (Error = {row.ERROR})")
    trace = traces[ traces['trace_id'] == row.Index ]
    labelled = model.predict(row.TPL_ID, trace )
    unexpected = labelled[ labelled['strangeness'] == 3 ]
    unexpected['color'] = unexpected['event'].apply(colorizer.tokenize)
    print()
    print( unexpected['color'].unique()[:20] )
    display( labelled.groupby('strangeness').size().reset_index(name='count'))


-------
Unexpected events in GRAVITY_gen_cal_init (Error = True)

['unforseen template error'
 'tacerr_mod_block unable to modify block parameters controlmet'
 'tacerr_block_name unknown block controlmet'
 'tacerr_send_request request to rtcontroller failed tacshmsg_modify_block'
 'modify block parameters controlmet red'
 'block parameters controlmet red']


,strangeness,count
0,0,226
1,1,3
2,2,2
3,3,6


-------
Unexpected events in GRAVITY_gen_tec_checkMetZero (Error = True)

['erradd parameter error is a null pointer errid {} module db location dbreaddirect c {}'
 'erradd parameter error is a null pointer errid {} module cai location caigetattributes c {}'
 'erradd parameter error is a null pointer errid {} module cai location cairwattribute c {}'
 'unforseen {} local alias lgvmet scanningstatus {} normal scan system alarm for device pointname check device status'
 'unforseen {} local alias lgvmet scanningstatus {} fatal scan system alarm for device pointname check device status'
 'wgv {} local alias lgvmet scanningstatus {} normal'
 'wgv {} local alias lgvmet scanningstatus {} fatal'
 'ccserr_scan_fct_call error occurred in function ccsscansend'
 'ccserr_db_quality point alias lcu lgvmet tacsimulation has bad data quality suspect'
 'seqerr_bad_quality alias lcu lgvmet tacsimulation has bad quality'
 'seqerr_bad_quality alias p881 running has bad quality'
 'ccserr_db_quality point al

,strangeness,count
0,0,1769
1,2,1
2,3,4032


-------
Unexpected events in GRAVITY_single_obs_exp (Error = True)

[]


,strangeness,count
0,-1,5490


-------
Unexpected events in GRAVITY_single_obs_exp (Error = True)

[]


,strangeness,count
0,-1,5032


-------
Unexpected events in GRAVITY_single_acq (Error = True)

['gravity_single_obs_calibrator single field calibrator yellow'
 'seq obsseq o s' 'det{} ndit object {}' 'seq comment no_comment'
 'ins spec res same' 'seq {} {}' 'det{} ndit {}' 'dpr catg calib'
 'ins spec pol same' 'current rmnrec status idle'
 'fringetrk loop state {}' 'initial instrument setup brown'
 'setup expoid {} noexposure function ins mode parallel blue'
 'setting sp mode to in med brown' 'observing sequence o s brown'
 'status function ins drot{} blue'
 'send command status ins drot{} false to sub system ics'
 'last reply to status from ics received ins drot{} id hwp{} {}ins drot{} name field rotator {} {}ins drot{} mode vlti {}in len {}'
 'ins drot{} id hwp{} ins drot{} name field rotator {} ins drot{} mode vlti ins drot{} posang {} ins drot{} ra {} ins drot{} dec {} ins drot{} pos'
 '{} ins drot{} modeangle {} springgreen4']


,strangeness,count
0,0,812
1,1,97
2,2,12
3,3,331
